In [ ]:
import openai
import os 
import requests
import shutil
from PIL import Image

with open('key.txt', 'r') as f:
    api_key = f.read().strip('\n')
    assert api_key.startswith('sk-'), 'error loading the API key'
    
openai.api_key = api_key

In [ ]:
def create_meals(ingredients, kcal=2000):
    prompt = f'''Create a healthy daily meal plan for breakfast, lunch and dinner based on the 
    following ingredients {ingredients}.
    Explain each recipe
    The total daily intake of calories should be below {kcal}.
    Assign a suggestive and concise title to each meal.
    Your answer should end with 'Titles: ' and the title of each recipe.
    '''
    
    messages = [
        {'role': 'system', 'content': 'You are a talented cook'},
        {'role': 'user', 'content': prompt}
    ]
    
    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=messages,
        temperature=1,
        max_tokens = 1024,
        n=1
    )
    
    return response.choices[0].message.content

foods = 'broccoli, chicken, fish, vegetables, cabbage, eggs, olive oil'
output = create_meals(foods)
print(output)

In [ ]:
titles = output.splitlines()[-3:]

titles = [t.strip('- ') for t in titles]
print(titles)

In [ ]:
def create_and_save_image(title, extra=''):
    image_prompt = f'{title}, {extra}, high quality food photography'
    
    response = openai.Image.create(
        prompt=image_prompt,
        n=1,
        size='1024x1024'
    )
    
    image_url = response.data[0].url
    
    image_resource = requests.get(image_url, stream=True)
    
    if image_resource.status_code == 200:
        image_filename = f'{title}.png'
        
        with open(image_filename, 'wb') as f:
            shutil.copyfileobj(image_resource.raw, f)
            return image_filename
    else:
        print('error accessing the image')
        return False
    
image_filename = create_and_save_image(titles[0], 'white background')

Image.open(image_filename)